# 堆排序(最小堆算法)

python库: heapq

这个模块实现了堆队列算法，即优先队列算法。

堆是一棵完全二叉树，其中每个节点的值都小于等于其各个子节点的值。这个使用数组的实现，索引从 0 开始，且对所有的 k 都有 $heap[k] <= heap[2*k+1]$ 和 $heap[k] <= heap[2*k+2]$。比较时不存在的元素被认为是无限大。堆最有趣的特性在于最小的元素总是在根结点：heap[0]。

创建最小堆: 空列表[], 或者使用heapify()函数将列表转换为最小堆

heapq定义了以下函数:
+ heapq.heappush(*heap, item*):
将item的值加入heap中, 同时保持**堆的不变性**
+ heapq.heappop
+ heapq.heappushpop(*heap, item*): 等价于先heappush再heappop, 更有效率
+ heapq.heapify(x)
+ heapq.heapreplace(*heap, item*)
弹出并返回heap中最小值, 同时push进新的item, 堆的大小不变. 如果堆为空则raise IndexError

该模块还提供了三个基于堆的通用目的的函数

+ heapq.merge(*iterables, key=None, reverse=False*)
类似sorted, 将多个**已排序**的输入和并为一个已排序的输出
+ heapq.nlargest(*n, iterable, key=None*)
+ heapq.nsmallest(*n, iterable, key=None*)
  



# 优先队列的实现

## 优先队列实现的挑战

+ 排序稳定性: 相同优先级任务按照最初被加入队列顺序返回
+ 如果prority相同且task之间未定义默认比较顺序, 则两个(priority, task) 元祖之间无法比较
+ 任务优先级发生改变的情况下, 如何将其移至堆中的新位置
+ 如果一个挂起的人物需要被删除, 如何找到他并一处队列
  
## 解决方案

针对前两项挑战的一种解决方案是引入条目计数来打破优先级的平局
```
counter = itertools.count()
count = next(counter)
entry = [priority, count, task]
```

两个task之间不可比的问题的另一种解决方案是-- 创建一个忽略task, 只比较priority字段的包装器类:

In [1]:
from dataclasses import dataclass, field
from typing import Any

@dataclass(order=True)
class PrioritizedItem:
    priority: int
    item: Any=field(compare=False)

In [2]:
item1 = PrioritizedItem(priority=10, item="Item 1")
item2 = PrioritizedItem(priority=5, item="Item 2")

其余的挑战主要包括找到挂起的任务并修改其优先级或将其完全移除。 找到一个任务可使用一个指向队列中条目的字典来实现。

移除条目或改变其优先级的操作实现起来更为困难，因为它会破坏堆结构不变量。 因此，一种可能的解决方案是将条目标记为已移除，再添加一个改变了优先级的新条目:

In [ ]:
import itertools
from heapq import *
pq = []                         # list of entries arranged in a heap
entry_finder = {}               # mapping of tasks to entries
REMOVED = '<removed-task>'      # placeholder for a removed task
counter = itertools.count()     # unique sequence count

def add_task(task, priority=0):
    'Add a new task or update the priority of an existing task'
    if task in entry_finder:
        remove_task(task)
    count = next(counter)
    entry = [priority, count, task]
    entry_finder[task] = entry
    heappush(pq, entry)

def remove_task(task):
    'Mark an existing task as REMOVED.  Raise KeyError if not found.'
    entry = entry_finder.pop(task)
    entry[-1] = REMOVED

def pop_task():
    'Remove and return the lowest priority task. Raise KeyError if empty.'
    while pq:
        priority, count, task = heappop(pq)
        if task is not REMOVED:
            del entry_finder[task]
            return task
    raise KeyError('pop from an empty priority queue')